### Example Exploratory Notebook

Use this notebook to explore the data generated by the pipeline in your preferred programming language.

**Note**: This notebook is not executed as part of the pipeline.

In [0]:
%sql
USE mgiglia.dev_matthew_giglia_redpanda;

In [0]:
%sql
select * from profiles_bronze ORDER BY timestamp DESC;

In [0]:
%sql
SELECT * FROM profiles ORDER BY timestamp DESC;

In [0]:
from pyspark.sql.functions import col, unix_millis

In [0]:
df = spark.table("mgiglia.dev_matthew_giglia_redpanda.profiles_bronze")
from pyspark.sql.functions import sha2, concat_ws
df = df.withColumn("recordId", sha2(concat_ws("||", *df.columns), 256))


In [0]:
display(df)

In [0]:
# Create a streaming DataFrame by reading from the profiles table
df = (
  spark.table("mgiglia.dev_matthew_giglia_redpanda.profiles_bronze")
    .select("topic", "partition", "offset", "value_str", "timestampType", "timestamp", "ingestTime")
    .withColumn(
      "latency_ms"
      ,unix_millis(col("ingestTime")) - unix_millis(col("timestamp"))
    )
    .orderBy(col("timestamp"), ascending=False)
    .limit(1000)
  )

# Display the streaming DataFrame
display(df)

In [0]:
# Calculate the average latency_ms from streaming_df
average_latency_ms_df = df.groupBy().avg("latency_ms")

# Display the result
display(average_latency_ms_df)

In [0]:
%sql
select count(*) from profiles;

In [0]:
%sql
DESCRIBE HISTORY profiles_bronze;

In [0]:
df = (
  spark.read.option("readChangeFeed", "true")
  .option("startingVersion", 11)
  .table(f"profiles_bronze")
  .filter("_change_type = 'delete'")
)
display(df)

In [0]:
%sql
DESCRIBE EXTENDED profiles AS JSON;

In [0]:
%sql 
FROM profiles_scd2 |>
AGGREGATE COUNT(*) as rcrd_cnt 
GROUP BY __START_AT._commit_version 